In [46]:
import pandas as pd
main = pd.DataFrame(pd.read_excel("data/main1010.xlsx"))

In [107]:
##CREATE SHIP

#若尚未創建過的船隻，建立船隻
shipIDlist = []
shiplist = []
tasklist = []
idcnt = 0
last_time = 0

for row in range(main.shape[0]):
    if last_time != sh.start_time: ##create new task
        ##create new ship
        if sh.sailing_status == 'I':
            ship = (SHIP(ship_id = int(sh.ship_no), cur_pos = (0, 0), weight = sh.total_weight)) #temp place
            tasklist.append(TASK(i = idcnt, ship = ship, task_state = 0, ship_state = 0, start_time = sh.start_time, start = int(sh.port)+9000, dest = int(sh.place2), side=sh.park))

        elif sh.sailing_status == 'O':
            ship = (SHIP(ship_id = int(sh.ship_no), cur_pos = get_pier_latlng(sh.place2), weight = sh.total_weight))
            tasklist.append(TASK(i = idcnt, ship = ship, task_state = 0, ship_state = 1, start_time = sh.start_time, start = int(sh.port)+9000, dest = int(sh.place2), side=sh.park))

        elif sh.sailing_status == 'T':
            ship = (SHIP(ship_id = int(sh.ship_no), cur_pos = get_pier_latlng(sh.place1), weight = sh.total_weight))
            tasklist.append(TASK(i = idcnt, ship = ship, task_state = 0, ship_state = 2, start_time = sh.start_time, start = int(sh.place1), dest = int(sh.place2), side=sh.park))

#         break
    else:
        continue

print(type(tasklist[0].start_time))



<class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [69]:
""" define the model ex. Ship, Tug, Task ... etc
"""
from enum import Enum, IntEnum
from datetime import time, datetime, timedelta
# from settings import SHIP_SPEED, TUG_SPEED

class Event(Enum):
    """All kinds of events which trigger planning algorithm

    Attributes:
        ROUTINE:            dispatch every one hour
        SHIP_DELAY:         delay caused by the ship
        TUG_DELAY:          delay caused by the tug
        UNIDENTIFIED_DELAY: delay occurs under unidentified situation
        MISMATCH:           the previous assignment is refused by the pilot
        BEFOREHAND:         any of the tugs finish task in advance
        CANCELED:           the task is canceled
    """

    ROUTINE = 'routine'
    CANCELED = 'canceled'

    TUG_DELAY = 'tug_delay'
    SHIP_DELAY = 'ship_delay'
    UNIDENTIFIED_DELAY = 'unidentified_delay'
    PIER_DELAY = 'pier_delay'

    MISMATCH = 'mismatch'
    BEFOREHAND = 'beforehand'
    
    ADD_TUG = 'add_tug'        


class TUG_STATE(Enum):
    """States of tugs which are considered when planning

    Attributes:
        FREE:        a tug which is ready to work immediately
        BUSY:        a tug which is busy working on a specific tesk
        UNAVAILABLE: a tug which is either broken or out of order
    """
    FREE = 0
    BUSY = 1
    UNAVAILABLE = 2


class CHARGE_TYPE(IntEnum):
    """Types of tugs which are used in charging and planning.
        Values of attributes correpond to weights according to the dispatching rule
    """

    TYPE_117 = 117
    TYPE_118 = 118
    TYPE_119 = 119
    TYPE_120 = 120
    TYPE_0 = 130


class TASK_STATE(IntEnum):
    """States of task describing
        if tugs are assigned (UNASSIGNED or ASSIGNED)
        if the task is finished (UNPROCESSED, PROCESSING, or PROCESSED)
    """

    UNPROCESSED_UNASSIGNED = 0
    UNPROCESSED_ASSIGNED = 1
    PROCESSING_ASSIGNED = 2
    PROCESSED = 3


class SIDE(Enum):
    LEFT = 'L'
    RIGHT = 'R'
    OPTIONAL = 'O'


class SHIP_STATE(Enum):
    IN = 0
    OUT = 1
    TRANSFER = 2

class TUG():
    """Main entities to be planned

    Attributes:
        id (int):           ID of the tug
        pos ((int, int)):   current position of the tug, a pair of latitude and lontitude
        type (CHARGE_TYPE): charging type
        hp (int):           horsepower of the tug
        velo (int):         velocity, default 10kt
        state (TUG_STATE):  current state of the tug
        next_available_time: if tug state is busy,we will recorded the time which it will finish the task
    """

    def __init__(self, tug_id, cur_pos, charge_type,
                hp, state=TUG_STATE.FREE, velo=10,next_available_time = datetime.now()):
        self.tug_id = tug_id
        self.pos = cur_pos
        self.type = charge_type
        self.hp = hp
        self.state = state
        self.velo = velo
        self.next_available_time = next_available_time

    def __str__(self):
        return("TUG {}: type = {} hp = {} state = {}".format(self.tug_id, self.type, self.hp, self.state))


# TUG list and TUG list 之間的比較
class CHARGE_TYPE_LIST(object):
    charge_type_list = []
    
    def __init__(self, charge_type_list):
        self.charge_type_list = charge_type_list
        

    def __eq__(self, other):
        myself = sorted(self.charge_type_list)
        other = sorted(other.charge_type_list)
        if len(myself) != len(other):
            return (False)
        else:
            for i in range(len(myself)):
                if myself[i] != other[i]:
                    return (False)
        return (True)
       
    def __gt__(self, other):
        myself = sorted(self.charge_type_list)
        other = sorted(other.charge_type_list)
        if len(myself) != len(other):
            return (False)
        else:
            for i in range(len(myself)):
                if myself[i] < other[i]:
                    return (False)
        return (True)

    def __ge__(self, other):
        return ((self > other) or (self == other))
      
    

class SHIP():
    """Characteristics of ships would be considered in planning

    Attributes:
        id (int):        ID of the ship
        pos ((int,int)): current position of the ship, a pair of latitude and lontitude
        weight (int):    ship weight
        velo (int):      velocity, default 10kt
    """

    def __init__(self, ship_id, cur_pos, weight, velo=10):
        self.ship_id = ship_id
        self.pos = cur_pos
        self.weight = weight
        self.velo = velo
    
    def __str__(self):
        return("SHIP {}: weight = {}".format(self.ship_id, self.weight))


class TASK():
    """
    Attributes:
    id (int):                  ID
    ship (SHIP):               ship
    task_state (TASK_STATE):   task state
    ship_state (SHIP_STATE):   ship state
    start_time (datetime):     starting time
    work_time (datetime) :     working time predicted by regression model
    start (int):               starting place, port number
    to (int):                  final place, port number
    side (SIDE):               a ship parks with its left or right side
    """

    # 這裡好像是一個static 的變數
    work_time = None
    tug = []
    task_state = TASK_STATE.UNPROCESSED_UNASSIGNED

    def __init__(self, i, ship, task_state, ship_state, start_time, start, dest, side='O'):
        self.id = i
        self.ship = ship
        self.ship_state = ship_state
        self.task_state = task_state
        self.start_time = start_time
        self.start = start
        self.to = dest
        self.side = side
    def __str__(self):
        return("TASK {}: ship {} goes {} from ({:.2f}, {:.2f}) to ({:.2f}, {:.2f}) at {} by {}".format(
                    self.id, self.ship.ship_id, self.ship_state, self.start[0],self.start[1], self.to[0], self.to[1], 
                    self.start_time.strftime("%Y-%m-%d %H:%M"), self.side.name))

    def show(self):
        res = "==== Task {} Information ====\n".format(self.id)
        res += "Ship ID: {}  Starting at: {}  Work time: {}  State: {}\n\n".format(self.ship.ship_id, 
        self.start_time, self.work_time, self.task_state)
        res += "========= Tug Set =========\n"
        for t in self.tug:
            res += "{} ".format(t.tug_id)
        res += "\n"
        print(res)


In [94]:
"""interface for access existing data
"""

# read excel data
import pandas as pd
# harbor to pier distance
df_port_to_pier = pd.read_excel("data/complete_dis.xlsx",index_col = "代號")
#pier to pier distance
df_pier_to_pier = pd.read_excel("data/complete_dis_meter.xlsx",index_col = 1)

#reverse: if 'L' then 0 順 1 逆
df_reverse1 = pd.read_excel("data/左靠逆靠.xlsx")


def get_portToPier_dist(port,pier):
    portnum = 9001 if port == 1 else 9002
    return (df_port_to_pier.loc[int(portnum),int(pier)])

def get_pierToPier_dist(pier1, pier2):
    return df_pier_to_pier.loc[int(pier1),int(pier2)]


def get_pier_latlng(pier):
    if pier == 1:
        return (22.616677,120.265942)
    elif pier == 2:
        return (22.552638,120.316716)
    return df_port_to_pier.loc[int(pier),"經緯度"]

def get_reverse(task):
    reverse = 0
    if task.ship_state == 'I':
        if task.start == 9001:
            if task.side == 'L':
                reverse = bool(df_reverse1.loc[int(task.to),9001])
            else:
                reverse = not(bool(df_reverse1.loc[int(task.to),9001]))
        elif task.start == 9002:
            if task.side == 'L':
                reverse = bool(df_reverse1.loc[int(task.to),9002])
            else:
                reverse = not(bool(df_reverse1.loc[int(task.to),9002]))
    elif task.ship_state == 'T':

        reverse = 0
    return reverse
